### 预处理：获取文件中的所有飞机名称

首先pandas读文件

In [1]:
import pandas as pd

original_file = pd.read_csv('D:\\Study\\Computing\\发射机与目标机判定\\data\\51st Bisons vs CNF Rd 1__1HZ.csv') # 读取csv文件
Ids = original_file['Id'].values # 读取Id列
names = [] # 创建空列表用于储存飞机名称

In [5]:
original_file.loc[original_file['Id'] == '102']['Type'].values[0]

'Air+FixedWing'

获取文件中的所有飞机名称

In [6]:
for Id in Ids:
    found = False
    type_name = original_file.loc[original_file['Id'] == Id]['Type'].values[0] # 读取Type列并获取第一个值
    if type_name.find('Air') == -1:
        # 查找所有带有“Air”的目标——即飞机，储存在names中
        # 去除导弹
        continue
    for name in names:
        # 去除重复的目标
        if Id == name:
            found = True
            break
    if not found:
        names.append(Id)


print(names)

['102', '202', '106', '10A', '109', '107', '108', '10C', '105', '10B', '104', '10E', '10F', '110']


## 现在开始处理问题

首先还是读文件

In [7]:
original_filename = '51st vs 36th R1__1HZ.csv'

ttmp = 'D:\\Study\\Computing\\发射机与目标机判定\\data\\' + original_filename # 读取csv文件

original_file = pd.read_csv(ttmp)
Ids = original_file['Id'].values
names = []

获取空战发生的时间区间

In [8]:
time_min = original_file['Unix time'].min() # 获取最小时间
time_max = original_file['Unix time'].max() # 获取最大时间

In [9]:
time_file_name = original_filename + '_time' + '.txt' # 时间文件名

df = pd.DataFrame(index=range(time_max - time_min + 30)) # 根据指定时间区间创建空DataFrame

和之前一样获取文件中的所有飞机名称

In [10]:
for Id in Ids:
    found = False
    type_name = original_file.loc[original_file['Id'] == Id]['Type'].values[0]
    if type_name.find('Air') == -1:
        continue
    for name in names:
        if Id == name:
            found = True
            break
    if not found:
        names.append(Id)

现在开始遍历names，依次分析各架飞机，首先对一架飞机而言，例如name = '102'

In [11]:
import numpy as np

name = '102' # 选择目标

acts = [] # 用于储存动作序列
file = original_file[original_file.Id == name] # 获取Id为name的目标的所有数据，是原数据集的子集

start_time = file.iloc[0, 1] - time_min # 该飞机开始被记录的时间

# 获取飞机状态
roll = file['Roll'].values
pitch = file['Pitch'].values
yaw = file['Yaw'].values
ground_distance = file['Altitude'].values

首先需要定义各种简单动作

In [12]:
from sklearn.linear_model import LinearRegression


def calc_b(y): 
    # R^2检验，返回值包括拟合斜率和线性拟合度，线性拟合度大于0.95则认为是线性关系，此时滑动窗口的长度不需要再减小
    reg = LinearRegression()
    len_ = len(y)
    y = y.reshape(-1, 1)
    x = [i for i in range(1, len_ + 1)]
    x = np.array(x)
    x = x.reshape(-1, 1)
    reg.fit(x, y)
    return reg.coef_, reg.score(x, y)


def decide_basic_action(yaw, pitch, roll, ground_dist):
    # 根据飞机状态判断基本动作

    # 计算各飞行参数的拟合斜率和线性拟合度
    yaw_b, yaw_r2 = calc_b(yaw)
    pitch_b, pitch_r2 = calc_b(pitch)
    roll_b, roll_r2 = calc_b(roll)
    ground_dist_b, ground_dist_r2 = calc_b(ground_dist)

    threshold = 0.95 # 线性拟合度阈值

    if yaw_r2 < threshold or pitch_r2 < threshold or roll_r2 < threshold or ground_dist_r2 < threshold:
        # 线性拟合度小于阈值，认为不是线性关系，返回0，需要减小滑动窗口长度
        return 0

    ans = [] # 储存基本动作

    if (yaw_b > -1) and (yaw_b < 1) and (ground_dist_b > -2) and (ground_dist_b < 2):
        ans.append(1) # 平飞：1
    if ground_dist_b > 2:
        ans.append(2) # 上升：2
    if ground_dist_b < -2:
        ans.append(3) # 下降：3
    if yaw_b > 1:
        ans.append(4) # 转弯：4
    if roll_b > 10:
        ans.append(5) # 翻滚：5
    if len(ans) == 0:
        return 0 # 无法检测，需要进一步缩小滑动窗口长度
    else:
        return ans

下面开始使用滑动窗口

In [13]:
end = len(roll) # 结束位置
cur = 0 # 当前位置
default_sliding_window_len = 10 # 默认滑动窗口长度

In [37]:
acts = [] # 储存基本动作
while cur < end: # 遍历时间区间
    begin = cur # 更新当前位置
    sliding_window_len = default_sliding_window_len # 初始设置为默认滑动窗口长度

    act = decide_basic_action(
        yaw = yaw[begin : begin+sliding_window_len], 
        pitch = pitch[begin : begin+sliding_window_len],
        roll = roll[begin : begin+sliding_window_len],
        ground_dist = ground_distance[begin : begin+sliding_window_len]
        )
    # 基于当前滑动窗口长度判断基本动作
        
    while act == 0: 
        # 无法检测，需要进一步缩小滑动窗口长度直至可以检测
        sliding_window_len -= 1 # 缩小滑动窗口长度
        act = decide_basic_action(
            yaw=yaw[begin:begin + sliding_window_len],
            pitch=pitch[begin:begin + sliding_window_len],
            roll=roll[begin:begin + sliding_window_len],
            ground_dist=ground_distance[begin:begin + sliding_window_len]
            )
        # 基于当前滑动窗口长度再次判断基本动作
            
    for i in range(sliding_window_len):
        tmp = ''
        for a in act:
            tmp += str(a) 
            # a = 1, 2, 3, 4, 5分别表示不同的基本动作
            # 例如tmp = '123'表示当前滑动窗口内的基本动作为平飞、上升、下降

        acts.append(tmp) # 记录基本动作序列
        # 例如acts = ‘123123123’表示此次滑动窗口长度为3，窗口内的3个时刻基本动作序列依次为：
        # 1：平飞、上升、下降，2：平飞、上升、下降，3：平飞、上升、下降

    cur += sliding_window_len # 更新当前位置

    df = df.astype('object') # 用于储存字符串格式的基本动作序列
    
    df[name] = '0' # 初始化该列，统一设置成未起飞状态：0
    df.loc[start_time: start_time+len(acts), name] = acts
    df.loc[start_time+len(acts):, name] = -1
    # 将acts嵌入df的一列，其中飞机起飞前设置为0，后续为基本动作序列，最后是占位的-1

现在使用循环处理所有飞机对象

In [ ]:
for name in names:
    acts = []
    file = original_file[original_file.Id == name]
    start_time = file.iloc[0, 1] - time_min
    roll = file['Roll'].values
    roll = np.array(roll)
    pitch = file['Pitch'].values
    pitch = np.array(pitch)
    yaw = file['Yaw'].values
    yaw = np.array(yaw)
    ground_distance = file['Altitude'].values
    ground_distance = np.array(ground_distance)

    start = 0
    end = len(roll)
    cur = 0
    default_sliding_window_len = 10

    while cur < end:
        begin = cur
        sliding_window_len = default_sliding_window_len
        act = decide_basic_action(
            yaw=yaw[begin:begin + sliding_window_len], 
            pitch=pitch[begin:begin + sliding_window_len],
            roll=roll[begin:begin + sliding_window_len],
            ground_dist=ground_distance[begin:begin + sliding_window_len]
            )
        
        while act == 0:
            sliding_window_len -= 1
            act = decide_basic_action(
                yaw=yaw[begin:begin + sliding_window_len],
                pitch=pitch[begin:begin + sliding_window_len],
                roll=roll[begin:begin + sliding_window_len],
                ground_dist=ground_distance[begin:begin + sliding_window_len]
                )
            
        for i in range(sliding_window_len):
            tmp = ''
            for a in act:
                tmp += str(a)
            acts.append(tmp)
        cur += sliding_window_len

        df = df.astype('object')
        df[name] = '0'
        df.loc[start_time: start_time+len(acts)-1, name] = acts
        df.loc[start_time+len(acts):, name] = -1

In [40]:
print(df)
tttmp = 'ques2_' + original_filename
df.to_csv(tttmp)

     102 202 104 105 107 106 10A 109 10D 108 10E 10C 10B 10F
0      3   3   0   0   0   0   0   0   0   0   0   0   0   0
1      3   3   0   0   0   0   0   0   0   0   0   0   0   0
2      3   3   0   0   0   0   0   0   0   0   0   0   0   0
3      3   3   0   0   0   0   0   0   0   0   0   0   0   0
4      1   1   0   0   0   0   0   0   0   0   0   0   0   0
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
5174  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
5175  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
5176  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
5177  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
5178  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1

[5179 rows x 14 columns]
